# Analysis on Personal Spending

### Import bank transactions
### Extrapolate into the future

notes: 
Opting to split between loan and transaction accounts believe the latter can be analysed to best analyse spending

In [103]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [104]:
#issues with two files (credit card + savingss) having values containing commas, breaking out the value to multiple columns and causing errors
#manually opening the CSV files and correctin - would be to automate the correctin of these catw
#ideally I'd export in a different format, or rpelace the commas before epxortin  but neither are possible in this case

#folder containing one or more csv file to import
inputfolder = 'C:\\Users\\mcleodb\\Documents\\SpendFiles 20210405\\'
os.chdir(inputfolder)
files = os.listdir()
templist = []
#import each file to temp dataframe and append to the main one - adding column to capture the source
for i in files:
    loc = inputfolder + i
    print('import file: ',loc)
    tempdf = pd.read_csv(loc)
    tempdf['sourcefile'] = i
    templist.append(tempdf)
    dfraw = pd.concat(templist)

import file:  C:\Users\mcleodb\Documents\SpendFiles 20210405\creditcard trans050421.csv
import file:  C:\Users\mcleodb\Documents\SpendFiles 20210405\export.csv
import file:  C:\Users\mcleodb\Documents\SpendFiles 20210405\loan fixed trans050421.csv
import file:  C:\Users\mcleodb\Documents\SpendFiles 20210405\loan variable trans050421.csv
import file:  C:\Users\mcleodb\Documents\SpendFiles 20210405\savings trans050421.csv


In [105]:
#data cleaning - formatting dates, and consolidating debit/credit to single column
dfraw['Debit_neg'] = dfraw['Debit'] * -1
dfraw['amount'] = dfraw[['Debit_neg','Credit']].sum(axis=1)
df_all = dfraw[['Description', 'amount', 'Category',
       'SubCategory', 'sourcefile']].copy() 
df_all['date'] = pd.to_datetime(dfraw['Date'],format=r'%Y%m%d')

#I like the idea of having a column to denote if charge is a debit or credit
df_all['charge'] = df_all.amount.apply(lambda x: 'debit' if x <= 0 else 'credit')
#filter after 1/5/20 to exclude some initial loan transfers which are not meaningful for this analysis
df_all = df[df['date'] >= '2020-05-01']
df.to_csv('export.csv')


In [106]:
df_spend = df[df['sourcefile'].str.match('sav')]
df_spend = df_spend.append(df[df['sourcefile'].str.match('credit')])

In [117]:
df_spend.set_index('date', inplace=True)    

In [126]:
df_spend.resample('W').rolling(5).sum().plot()

AttributeError: 'DatetimeIndexResampler' object has no attribute 'rolling'